<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Train-a-bi-LSTM-to-predict-short-entities" data-toc-modified-id="Train-a-bi-LSTM-to-predict-short-entities-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Train a bi-LSTM to predict short entities<br></a></span><ul class="toc-item"><li><span><a href="#Set-up-" data-toc-modified-id="Set-up--1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Set up-</a></span><ul class="toc-item"><li><span><a href="#Import-necessry-packages" data-toc-modified-id="Import-necessry-packages-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Import necessry packages</a></span></li><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Helper functions</a></span></li><li><span><a href="#Load-in-data" data-toc-modified-id="Load-in-data-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Load in data</a></span><ul class="toc-item"><li><span><a href="#Read-in-the-BioSample-annotations-and-build-dataframe" data-toc-modified-id="Read-in-the-BioSample-annotations-and-build-dataframe-1.1.3.1"><span class="toc-item-num">1.1.3.1&nbsp;&nbsp;</span>Read in the BioSample annotations and build dataframe</a></span></li><li><span><a href="#Read-in-metadata-on-BioSample-entries-that-will-allow-us-to-cap-on-study" data-toc-modified-id="Read-in-metadata-on-BioSample-entries-that-will-allow-us-to-cap-on-study-1.1.3.2"><span class="toc-item-num">1.1.3.2&nbsp;&nbsp;</span>Read in metadata on BioSample entries that will allow us to cap on study</a></span></li><li><span><a href="#Read-in-embeddings" data-toc-modified-id="Read-in-embeddings-1.1.3.3"><span class="toc-item-num">1.1.3.3&nbsp;&nbsp;</span>Read in embeddings</a></span></li><li><span><a href="#Import-merging-of-entities-from-file" data-toc-modified-id="Import-merging-of-entities-from-file-1.1.3.4"><span class="toc-item-num">1.1.3.4&nbsp;&nbsp;</span>Import merging of entities from file</a></span></li></ul></li></ul></li><li><span><a href="#Data-processing" data-toc-modified-id="Data-processing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data processing</a></span><ul class="toc-item"><li><span><a href="#Filter-training-set" data-toc-modified-id="Filter-training-set-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Filter training set</a></span><ul class="toc-item"><li><span><a href="#Filter-out-non-valid-attributes" data-toc-modified-id="Filter-out-non-valid-attributes-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>Filter out non-valid attributes</a></span></li><li><span><a href="#Cap-at-100-samples-per-study-to-avoid-study-bias" data-toc-modified-id="Cap-at-100-samples-per-study-to-avoid-study-bias-1.2.1.2"><span class="toc-item-num">1.2.1.2&nbsp;&nbsp;</span>Cap at 100 samples per study to avoid study bias</a></span></li><li><span><a href="#Filter-out-unwanted-training-examples-and-replace-all-white-space-with-'-'" data-toc-modified-id="Filter-out-unwanted-training-examples-and-replace-all-white-space-with-'-'-1.2.1.3"><span class="toc-item-num">1.2.1.3&nbsp;&nbsp;</span>Filter out unwanted training examples and replace all white space with ' '</a></span></li><li><span><a href="#Take-only-values-between-length-2-and-7" data-toc-modified-id="Take-only-values-between-length-2-and-7-1.2.1.4"><span class="toc-item-num">1.2.1.4&nbsp;&nbsp;</span>Take only values between length 2 and 7</a></span></li><li><span><a href="#Filter-out-test-set-examples" data-toc-modified-id="Filter-out-test-set-examples-1.2.1.5"><span class="toc-item-num">1.2.1.5&nbsp;&nbsp;</span>Filter out test set examples</a></span></li><li><span><a href="#Replace-whitespace-with-'-'" data-toc-modified-id="Replace-whitespace-with-'-'-1.2.1.6"><span class="toc-item-num">1.2.1.6&nbsp;&nbsp;</span>Replace whitespace with ' '</a></span></li><li><span><a href="#Final-dataset" data-toc-modified-id="Final-dataset-1.2.1.7"><span class="toc-item-num">1.2.1.7&nbsp;&nbsp;</span>Final dataset</a></span></li></ul></li><li><span><a href="#Train/val-split" data-toc-modified-id="Train/val-split-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Train/val split</a></span><ul class="toc-item"><li><span><a href="#Start-by-pulling-out-only-samples-that-are-in-our-dataset-from-technical-metadata,-that-has-studies-in-it" data-toc-modified-id="Start-by-pulling-out-only-samples-that-are-in-our-dataset-from-technical-metadata,-that-has-studies-in-it-1.2.2.1"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>Start by pulling out only samples that are in our dataset from technical metadata, that has studies in it</a></span></li><li><span><a href="#Get-unique-studies" data-toc-modified-id="Get-unique-studies-1.2.2.2"><span class="toc-item-num">1.2.2.2&nbsp;&nbsp;</span>Get unique studies</a></span></li><li><span><a href="#Select-studies-to-use-to-train-and-validate" data-toc-modified-id="Select-studies-to-use-to-train-and-validate-1.2.2.3"><span class="toc-item-num">1.2.2.3&nbsp;&nbsp;</span>Select studies to use to train and validate</a></span></li></ul></li><li><span><a href="#Generate-training-and-val-dataframes" data-toc-modified-id="Generate-training-and-val-dataframes-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Generate training and val dataframes</a></span><ul class="toc-item"><li><span><a href="#Training-set" data-toc-modified-id="Training-set-1.2.3.1"><span class="toc-item-num">1.2.3.1&nbsp;&nbsp;</span>Training set</a></span></li><li><span><a href="#Validation-set" data-toc-modified-id="Validation-set-1.2.3.2"><span class="toc-item-num">1.2.3.2&nbsp;&nbsp;</span>Validation set</a></span></li></ul></li></ul></li><li><span><a href="#Train-the-model" data-toc-modified-id="Train-the-model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Train the model</a></span><ul class="toc-item"><li><span><a href="#Model-set-up" data-toc-modified-id="Model-set-up-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Model set-up</a></span></li></ul></li><li><span><a href="#Analyze-current-model" data-toc-modified-id="Analyze-current-model-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Analyze current model</a></span><ul class="toc-item"><li><span><a href="#Figure-4B:-Accuracy-plot" data-toc-modified-id="Figure-4B:-Accuracy-plot-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span><font color="red">Figure 4B:</font> Accuracy plot</a></span></li><li><span><a href="#Model-accuracy" data-toc-modified-id="Model-accuracy-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>Model accuracy</a></span><ul class="toc-item"><li><span><a href="#Overall-accuracy" data-toc-modified-id="Overall-accuracy-1.4.2.1"><span class="toc-item-num">1.4.2.1&nbsp;&nbsp;</span>Overall accuracy</a></span></li><li><span><a href="#Figure-4C:-Per-class-accuracy" data-toc-modified-id="Figure-4C:-Per-class-accuracy-1.4.2.2"><span class="toc-item-num">1.4.2.2&nbsp;&nbsp;</span><font color="red">Figure 4C:</font> Per class accuracy</a></span></li></ul></li><li><span><a href="#Check-correct-vs-incorrect-probabilities" data-toc-modified-id="Check-correct-vs-incorrect-probabilities-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>Check correct vs incorrect probabilities</a></span></li><li><span><a href="#Performance-metrics" data-toc-modified-id="Performance-metrics-1.4.4"><span class="toc-item-num">1.4.4&nbsp;&nbsp;</span>Performance metrics</a></span><ul class="toc-item"><li><span><a href="#Figure-4D:-ROC-curve" data-toc-modified-id="Figure-4D:-ROC-curve-1.4.4.1"><span class="toc-item-num">1.4.4.1&nbsp;&nbsp;</span><font color="red">Figure 4D:</font> ROC curve</a></span></li><li><span><a href="#Figure-4A:-Metrics-table" data-toc-modified-id="Figure-4A:-Metrics-table-1.4.4.2"><span class="toc-item-num">1.4.4.2&nbsp;&nbsp;</span><font color="red">Figure 4A:</font> Metrics table</a></span></li></ul></li></ul></li></ul></li></ul></div>

# Train a bi-LSTM to predict short entities<br>
Adam Klie<br>
11/17/2019<br>
Script to train a model based on merged entities passed in and training set decided on

## Set up-

### Import necessry packages

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import spacy
import tensorflow
import keras
from keras.models import Sequential, model_from_json, load_model
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
from sklearn import preprocessing, metrics

/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
Using TensorFlow backend.


In [68]:
from matplotlib import rcParams
import matplotlib as mpl

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 600 
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 18
rcParams['patch.edgecolor'] = 'white'
rcParams['font.family'] = 'StixGeneral'


def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

### Helper functions

In [4]:
# Function to build LSTM model.
# embeddings: an mxn matrix of word embeddings with m words and n features for each word (this case is 5443656 x 200)
# shape: parameter split into number of hidden units, 
def compile_lstm(embeddings, shape, settings):  # function definition
    model = Sequential()
    model.add(Embedding(embeddings.shape[0],
                        embeddings.shape[1],
                        input_length=shape['max_length'],
                        trainable=False,
                        weights=[embeddings],
                        mask_zero=True))
    
    #the same dense layer is first applied extract the most useful info from embedding layers
    model.add(TimeDistributed(Dense(shape['nr_hidden'], use_bias=False)))
    model.add(Bidirectional(LSTM(shape['nr_hidden'],
                                 recurrent_dropout=settings['dropout'],
                                 dropout=settings['dropout'])))
    model.add(Dense(shape['nr_class'], activation='sigmoid'))
    model.compile(optimizer=Adam(lr=settings['lr']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def get_features(docs, max_length):
    docs = list(docs)
    Xs = np.zeros((len(docs), max_length), dtype='int32')
    for i, doc in tqdm(enumerate(docs),total=len(docs)):
        j = 0
        for token in doc:
            ##rever to word vector
            vector_id = token.vocab.vectors.find(key=token.orth)
            if vector_id >= 0:
                Xs[i, j] = vector_id
            else:
                Xs[i, j] = 0
            j += 1
            if j >= max_length:
                break
    return Xs

### Load in data

#### Read in the BioSample annotations and build dataframe

In [5]:
# Read in pandas Series from file
SRS_dir = "../data/allSRS_05_15_2018.pickle"
allSRS = pd.read_pickle(SRS_dir)

#### Read in metadata on BioSample entries that will allow us to cap on study

In [6]:
sra_dump_pickle_dir = '../data/sra_dump.pickle'
technical_meta_data_df = pd.read_pickle(sra_dump_pickle_dir)

#### Read in embeddings

In [7]:
nlp = spacy.load('../data/wikipedia-pubmed-and-PMC-w2v')

/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


#### Import merging of entities from file

In [9]:
model_iter = '11_class'

In [12]:
grouping_df = pd.read_csv('../results/embedding/{model}_entity_merging.csv'.format(model = model_iter))
myAttribs = grouping_df.attribute.unique()

## Data processing

### Filter training set

#### Filter out non-valid attributes

In [13]:
attribute_m = allSRS.index.get_level_values(1).isin(myAttribs)

In [14]:
subset_SRS = allSRS[attribute_m]

#### Cap at 100 samples per study to avoid study bias

In [15]:
max_sample_per_study_n = 100
capped_samples = technical_meta_data_df.groupby('Study').head(n = max_sample_per_study_n)['Sample']
print("Number of samples left after capping: %d" % (len(capped_samples)))

Number of samples left after capping: 2348023


In [16]:
capped_m = subset_SRS.index.get_level_values(0).isin(capped_samples.values)

In [17]:
subset_SRS = subset_SRS[capped_m]

#### Filter out unwanted training examples and replace all white space with ' '

In [18]:
filterTextL = ['not collected','not applicable','missing','n[/]?a','unknown', '-', '--', 'none', 'no']
filterTextRegex = "|".join(map(lambda myStr:'(?:{})'.format(myStr), filterTextL))

In [19]:
regex_m = subset_SRS.str.contains(filterTextRegex, case=False)

In [20]:
subset_SRS = subset_SRS[~regex_m]

#### Take only values between length 2 and 7

In [21]:
attribute_wc = subset_SRS.str.count(' ') + 1
wc_m = (attribute_wc >= 2) & (attribute_wc <= 7)

In [22]:
subset_SRS = subset_SRS[wc_m]

#### Filter out test set examples

In [26]:
with open('../results/validation/{model}/validation_SRSs.txt'.format(model=model_iter), 'r') as f:
    test_srs = [line.rstrip('\n') for line in f.readlines()]

In [27]:
test_m = ~subset_SRS.index.get_level_values(0).isin(test_srs)

In [28]:
subset_SRS = subset_SRS[test_m]

#### Replace whitespace with ' '

In [29]:
subset_SRS = subset_SRS.str.replace('\s+', ' ')

#### Final dataset

In [30]:
subset_df = pd.DataFrame(subset_SRS).reset_index()
subset_df.columns = ['srs', 'attribute', 'value']

In [31]:
subset_df['original_attribute'] = subset_df['attribute']
AttribToGroupNameS = grouping_df.groupby('attribute')['GroupName'].first()
subset_df['attribute'] = AttribToGroupNameS[subset_df['original_attribute'].values].values

In [32]:
subset_df.head()

,srs,attribute,value,original_attribute
0,SRS1024493,Species,Camellia oleifera,SCIENTIFIC_NAME
1,SRS568274,Species,Homo sapiens,nat-host
2,ERS1412828,Species,Pundamilia nyererei,SCIENTIFIC_NAME
3,ERS1412828,Species,Pundamilia nyererei,ArrayExpress-Species
4,SRS1219231,Species,Salmonella enterica subsp. enterica,SCIENTIFIC_NAME


### Train/val split

#### Start by pulling out only samples that are in our dataset from technical metadata, that has studies in it

In [34]:
valid_srs_m = allSRS.index.get_level_values(0).unique()
technical_meta_data_df_sub = technical_meta_data_df[technical_meta_data_df.Sample.isin(valid_srs_m)]

#### Get unique studies

In [35]:
study_s = technical_meta_data_df_sub['Study'].drop_duplicates()  # Get unique studies that can be found in allSRS
num_studies = len(study_s)

#### Select studies to use to train and validate

In [36]:
train_test_ratio = 0.8
train_n = int(num_studies * train_test_ratio)  # Number of training studies
train_studies = study_s.sample(n = train_n, random_state = 0).values  # pick those training studies randomly

In [37]:
# Label these studies with a column
technical_meta_data_df_sub['Train'] = technical_meta_data_df_sub['Study'].isin(train_studies).values  

/cellar/users/aklie/miniconda3/envs/deep_nlp_cpu/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [38]:
# Get the ids for the training and test samples
train_mask = technical_meta_data_df_sub['Train']
train_samples = technical_meta_data_df_sub['Sample'][train_mask].values  # IDs for training set
val_samples = technical_meta_data_df_sub['Sample'][~train_mask].values  # IDs for test set

### Generate training and val dataframes

In [39]:
nDupTextMax = 1000
cap_size = 20000

#### Training set

In [40]:
train_df = subset_df[subset_df.srs.isin(train_samples)]

In [41]:
# Cap how many of a given entity we want to use
train_df = train_df.sample(train_df.shape[0])
dedup_train_df = train_df.groupby(['value']).head(n = nDupTextMax)
dedup_train_df['value'].value_counts().head()

Wild type                   1000
Whole blood                 1000
Salmonella enterica         1000
Streptococcus agalactiae    1000
Macaca mulatta              1000
Name: value, dtype: int64

In [42]:
# Balance the classes with class cap
final_train_df = dedup_train_df.sample(dedup_train_df.shape[0]).groupby('attribute').head(n = cap_size)

In [43]:
print("The number of training examples is %d" % (final_train_df.shape[0]))
display(final_train_df.attribute.value_counts())

The number of training examples is 151811


Species              20000
Strain               20000
Cell type            20000
Age                  20000
Tissue               20000
Protocol             17202
Genotype             15033
Platform              7550
Condition/Disease     6698
Data type             3267
Sex                   2061
Name: attribute, dtype: int64

#### Validation set

In [51]:
val_df = subset_df[subset_df.srs.isin(val_samples)]

In [52]:
val_df = val_df.sample(val_df.shape[0])
val_df = subset_df[subset_df.srs.isin(val_samples)]
dedup_val_df = val_df.groupby(['value']).head(n = nDupTextMax)

In [53]:
final_val_df = dedup_val_df.sample(dedup_val_df.shape[0]).groupby('attribute').head(n = cap_size)

In [54]:
display(final_val_df.attribute.value_counts())

Species              20000
Cell type            20000
Tissue               20000
Age                   9328
Strain                7944
Protocol              4842
Genotype              4745
Condition/Disease     1963
Platform              1612
Data type             1552
Sex                    854
Name: attribute, dtype: int64

In [55]:
# Actual training and test split
final_train_df.shape[0]/(final_train_df.shape[0] + final_val_df.shape[0])

0.6205206600422642

In [57]:
final_train_df.to_pickle('../results/training/{model}/training_examples.pickle'.format(model = model_iter))
final_val_df.to_pickle('../results/training/{model}/test_examples.pickle'.format(model = model_iter))

## Train the model

### Model set-up

In [66]:
nr_hidden = 64 
max_length = 7 # 95% percentile of training phrase length from NCIT
dropout = 0.5
learn_rate = 0.001 # General NN config
nb_epoch = 1
batch_size = 100

In [59]:
classNames = AttribToGroupNameS.unique()
classNames

array(['Age', 'Genotype', 'Species', 'Strain', 'Cell type', 'Sex',
       'Platform', 'Condition/Disease', 'Protocol', 'Data type', 'Tissue'],
      dtype=object)

In [60]:
le = preprocessing.LabelEncoder()
le.fit(classNames)
nr_classes=len(le.classes_)

In [61]:
lstm_shape={'nr_hidden': 64, 'max_length': max_length, 'nr_class': nr_classes}
lstm_settings={'dropout': 0.5, 'lr': 0.001}

In [62]:
embeddings = nlp.vocab.vectors.data

(5443656, 200)

In [63]:
train_texts = final_train_df.value.tolist()
dev_texts = final_val_df.value.tolist()

train_labels = keras.utils.to_categorical(le.transform(final_train_df.attribute.values))
dev_labels = keras.utils.to_categorical(le.transform(final_val_df.attribute.values))

train_docs = list(tqdm(nlp.pipe(train_texts,n_threads=32),total=len(train_texts)))
dev_docs = list(tqdm(nlp.pipe(dev_texts,n_threads=32),total=len(dev_texts)))

train_X = get_features(train_docs, lstm_shape['max_length'])
dev_X = get_features(dev_docs, lstm_shape['max_length'])

100%|██████████| 92840/92840 [00:00<00:00, 163000.34it/s]


In [64]:
np.save('../results/training/{model}/classes.npy'.format(model = model_iter), le.classes_)

In [65]:
model = compile_lstm(embeddings, lstm_shape, lstm_settings)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 200)            1088731200
_________________________________________________________________
time_distributed_1 (TimeDist (None, 7, 64)             12800     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_2 (Dense)              (None, 11)                1419      
Total params: 1,088,811,467
Trainable params: 80,267
Non-trainable params: 1,088,731,200
_________________________________________________________________


In [67]:
%%capture keras_stdout
lstm = model.fit(train_X, train_labels, validation_data = (dev_X, dev_labels),
          nb_epoch = nb_epoch, verbose = 1, batch_size = batch_size)

In [72]:
import datetime
x = datetime.datetime.now()
today = x.strftime("%Y_%m_%d")
today

'2020_07_01'

In [75]:
lstm.model.save('../models/{model}_{date}.h5'.format(model = model_iter, date=today))

In [77]:
keras_stdout_str = keras_stdout.stdout
textfile = open('../results/training/{model}/keras_out.txt'.format(model = model_iter), 'w')
textfile.write(keras_stdout_str)
textfile.close()

## Analyze current model

### <font color=red>Figure 4B:</font> Accuracy plot

In [83]:
trainingAcc = pd.Series(keras_stdout_str.split('\r')).str.replace('\x08','').str.extract('acc: (\d+.\d+)').dropna()[0].astype(float)

In [84]:
trainingAcc.index = trainingAcc.index*batch_size/1000

In [101]:
fig,ax = plt.subplots(figsize=(4,3))
trainingAcc.plot(ax=ax)
ax.set_xlabel('# of training examples (thousands)')
ax.set_ylabel('Accuracy')
ax.set_yticks(np.arange(0.1,1.1,0.1))
ax.get_xaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.savefig('../doc/figures/Figure4/Figure4B.eps', dpi=600, bbox_inches="tight")
plt.savefig('../doc/figures/Figure4/Figure4B.png', dpi=600, bbox_inches="tight")
plt.close()

### Model accuracy

#### Overall accuracy

In [87]:
val_docs = list(tqdm(nlp.pipe(dev_texts,),total=len(dev_texts)))
val_X = get_features(val_docs,lstm_shape['max_length'])
predictM = lstm.model.predict_proba(val_X)

100%|██████████| 92840/92840 [00:00<00:00, 139228.91it/s]


In [88]:
probaDf_multI = pd.DataFrame(data = predictM, columns = le.classes_)
probaDf_multI.index = pd.MultiIndex.from_arrays([final_val_df.attribute.values, dev_texts], names=['entity','text'])
probaDf = probaDf_multI.reset_index()

In [89]:
myClasses = grouping_df.GroupName.unique()
predicted_types = probaDf.loc[:, probaDf.columns.isin(myClasses)].idxmax(axis=1)

In [90]:
print ('Accuracy:', ((probaDf.entity == predicted_types).mean()))

Accuracy: 0.8414476518741921


#### <font color =red>Figure 4C:</font> Per class accuracy

In [91]:
inEvalDf = probaDf_multI

In [92]:
minProbThreshold = 0.2
confidence_mask = inEvalDf.max(axis=1)>=minProbThreshold
tmpDf3 = inEvalDf.idxmax(axis=1).reset_index(name='predicted')

In [93]:
continDf=tmpDf3.groupby(['predicted','entity']).size().unstack().fillna(0)

In [94]:
continDf=continDf.loc[le.classes_, le.classes_]
continDf.index=continDf.index.copy()
continDf.columns=continDf.columns.copy()
continDf.index.name='Predicted'
continDf.columns.name='Actual'
percentDf=(continDf/continDf.sum(axis=0)).T

In [102]:
fig,ax=plt.subplots(figsize=(20,10))
g.data2d.columns=g.data2d.columns.copy()
g.data2d.index=g.data2d.index.copy()

sns.set(font_scale=2)
sns.heatmap(data=percentDf.loc[g.data2d.columns,g.data2d.columns],annot=True,fmt='0.0%', annot_kws={"fontsize":20})
ax.set_ylabel('Ground truth', fontsize = 24)
ax.set_xlabel('Predicted', fontsize = 24)

plt.savefig('../doc/figures/Figure4/Figure4C.eps', dpi=600, bbox_inches="tight")
plt.savefig('../doc/figures/Figure4/Figure4C.png', dpi=600, bbox_inches="tight")
plt.close();

### Check correct vs incorrect probabilities

In [103]:
incorrect_df = probaDf[probaDf.entity != predicted_types][['entity','text']]
incorrect_df['predicted'] = predicted_types

In [104]:
contdf_long=percentDf.stack().reset_index(name='Percent')

In [105]:
contdf_wrong=contdf_long[contdf_long.Actual!=contdf_long.Predicted].sort_values('Percent')
contdf_wrong[contdf_wrong.Percent>0.05]

,Actual,Predicted,Percent
107,Strain,Species,0.063444
67,Protocol,Cell type,0.072697
28,Condition/Disease,Protocol,0.073357
32,Condition/Disease,Tissue,0.093734
53,Genotype,Strain,0.110432
111,Tissue,Cell type,0.131000
21,Cell type,Tissue,0.215600
61,Platform,Protocol,0.236352


In [106]:
plt.figure(figsize=(16, 16))
predDf=probaDf_multI.stack().reset_index()
predDf.columns=['entity','text','predicted','proba']
predDf['match'] = (predDf.entity == predDf.predicted)
g =sns.boxplot(data=predDf,x='entity',y='proba', hue='match', showfliers=False)
g.set_xticklabels(g.get_xticklabels(), rotation=45)
predDf[predDf['match']].groupby(['entity'])['proba'].quantile(0.25).to_dict()

{'Age': 0.3259803056716919,
 'Cell type': 0.024990377947688103,
 'Condition/Disease': 0.05113299936056137,
 'Data type': 0.7079949975013733,
 'Genotype': 0.0750037282705307,
 'Platform': 0.28559234738349915,
 'Protocol': 0.019734062254428864,
 'Sex': 0.8908115029335022,
 'Species': 0.13601494580507278,
 'Strain': 0.01868274249136448,
 'Tissue': 0.04623797535896301}

### Performance metrics

#### <font color =red>Figure 4D:</font> ROC curve

In [110]:
fig,ax=plt.subplots(figsize=(16,16))
for myClass in le.classes_: 
    fpr,tpr,_= metrics.roc_curve((probaDf.entity==myClass),probaDf[myClass])    
    ax.plot(fpr,tpr,label="{myClass}\n  (AUC: {AUC})".format(myClass=myClass,AUC=str(metrics.auc(fpr,tpr))[:5]))
ax.set_xlabel('False positive rate', fontsize = 24)
ax.set_ylabel('True positive rate', fontsize = 24)
ax.legend(fontsize=20)
plt.savefig('../doc/figures/Figure4/Figure4D.eps', dpi=600, bbox_inches="tight")
plt.savefig('../doc/figures/Figure4/Figure4D.png', dpi=600, bbox_inches="tight")
plt.close();

#### <font color =red>Figure 4A:</font> Metrics table

In [120]:
inTmpDf=inEvalDf.idxmax(axis=1).reset_index(name='predicted')

In [121]:
inTmpDf['correct']=inTmpDf.entity==inTmpDf.predicted

In [122]:
inTmpDf.groupby('entity')['correct'].mean().sort_values()

entity
Cell type            0.674250
Platform             0.722084
Condition/Disease    0.780438
Genotype             0.817492
Strain               0.822633
Protocol             0.824040
Tissue               0.829850
Sex                  0.955504
Species              0.972400
Age                  0.977916
Data type            1.000000
Name: correct, dtype: float64

In [129]:
p_r_f1 = metrics.precision_recall_fscore_support(y_true=inTmpDf['entity'],y_pred=inTmpDf['predicted'],average='weighted')
metric_df = pd.DataFrame({'Accuracy':lstm.history['val_acc'], 
                          'Precision':p_r_f1[0], 
                          'Recall':p_r_f1[1], 
                          'F1 Score':p_r_f1[2]})
metric_df.to_csv('../doc/figures/Figure4/Figure4A.csv')
metric_df.to_csv('../results/training/{model}/metrics.csv'.format(model=model_iter))